In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgbm
import optuna

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

In [4]:
train = pd.read_csv('../data/tabular-playground-series-aug-2021/train.csv', index_col='id')

In [5]:
test = pd.read_csv('../data/tabular-playground-series-aug-2021/test.csv', index_col='id')

In [12]:
X = train.drop('loss', axis=1)
y = train.loss

In [25]:
def gradient_boost(boost):
    score = cross_val_score(boost, X, y, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=5, verbose=10)
    print(score.mean())
    return boost.fit(X, y)

In [33]:
def objective_lgbm(trial):
    params = {
        'objective': 'rmse',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_threads': 4,
        'feature_pre_filter': trial.suggest_categorical('feature_pre_filter', [True, False]),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'num_leaves': trial.suggest_int('num_leaves', 5, 50),
        'num_trees': trial.suggest_int('num_trees', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 1.0), 
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100)}
    
    lg = lgbm.LGBMRegressor(**params)
    
    score = cross_val_score(lg, X, y, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=3)
    rmse = score.mean()
    return rmse

In [34]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_lgbm, n_trials=100)
print(study.best_trial)

[I 2021-08-10 10:16:39,778] A new study created in memory with name: no-name-28c247f5-94c8-4d4e-b2b2-2b6556114aca
[I 2021-08-10 10:16:52,441] Trial 0 finished with value: -7.861281856947074 and parameters: {'feature_pre_filter': True, 'lambda_l1': 3.7163829027737454, 'lambda_l2': 4.7177508612934425, 'num_leaves': 13, 'num_trees': 369, 'learning_rate': 0.03610742106224141, 'feature_fraction': 0.5515103965834554, 'min_data_in_leaf': 100}. Best is trial 0 with value: -7.861281856947074.
[I 2021-08-10 10:17:21,577] Trial 1 finished with value: -7.915529848437409 and parameters: {'feature_pre_filter': True, 'lambda_l1': 4.101354592785546, 'lambda_l2': 1.8749590174207493, 'num_leaves': 35, 'num_trees': 713, 'learning_rate': 0.11572876838270583, 'feature_fraction': 0.944395426639909, 'min_data_in_leaf': 75}. Best is trial 0 with value: -7.861281856947074.
[I 2021-08-10 10:17:32,212] Trial 2 finished with value: -7.85999568378677 and parameters: {'feature_pre_filter': False, 'lambda_l1': 0.216

[I 2021-08-10 10:23:57,678] Trial 21 finished with value: -7.859990392424638 and parameters: {'feature_pre_filter': False, 'lambda_l1': 2.1247282176266267, 'lambda_l2': 2.2887553601238007, 'num_leaves': 27, 'num_trees': 479, 'learning_rate': 0.06939182456594292, 'feature_fraction': 0.7382314542620826, 'min_data_in_leaf': 33}. Best is trial 9 with value: -7.854982945253259.
[I 2021-08-10 10:24:13,402] Trial 22 finished with value: -7.857097000026935 and parameters: {'feature_pre_filter': False, 'lambda_l1': 2.5171382901592523, 'lambda_l2': 3.331378628092181, 'num_leaves': 22, 'num_trees': 501, 'learning_rate': 0.059857894087878935, 'feature_fraction': 0.6922018607141965, 'min_data_in_leaf': 35}. Best is trial 9 with value: -7.854982945253259.
[I 2021-08-10 10:24:24,069] Trial 23 finished with value: -7.936456954792629 and parameters: {'feature_pre_filter': False, 'lambda_l1': 3.457831913330623, 'lambda_l2': 3.216548994639407, 'num_leaves': 44, 'num_trees': 253, 'learning_rate': 0.196748

[I 2021-08-10 10:32:36,168] Trial 42 finished with value: -7.852418320964034 and parameters: {'feature_pre_filter': True, 'lambda_l1': 0.25326521459946416, 'lambda_l2': 4.318798135312507, 'num_leaves': 45, 'num_trees': 951, 'learning_rate': 0.011098880463625078, 'feature_fraction': 0.32101701170355773, 'min_data_in_leaf': 92}. Best is trial 31 with value: -7.8462917069433304.
[I 2021-08-10 10:32:56,729] Trial 43 finished with value: -7.848618499958838 and parameters: {'feature_pre_filter': True, 'lambda_l1': 0.5614439696666467, 'lambda_l2': 4.993772204757525, 'num_leaves': 42, 'num_trees': 813, 'learning_rate': 0.03563020149385601, 'feature_fraction': 0.23694078502792226, 'min_data_in_leaf': 91}. Best is trial 31 with value: -7.8462917069433304.
[I 2021-08-10 10:33:31,049] Trial 44 finished with value: -7.846841179552794 and parameters: {'feature_pre_filter': True, 'lambda_l1': 0.30231064803922436, 'lambda_l2': 4.723372871225082, 'num_leaves': 45, 'num_trees': 998, 'learning_rate': 0.0

[I 2021-08-10 10:41:55,179] Trial 64 finished with value: -7.8476391275242845 and parameters: {'feature_pre_filter': True, 'lambda_l1': 0.3419456921201233, 'lambda_l2': 3.4270751723993995, 'num_leaves': 46, 'num_trees': 969, 'learning_rate': 0.016287060688677295, 'feature_fraction': 0.2938707464912069, 'min_data_in_leaf': 94}. Best is trial 31 with value: -7.8462917069433304.
[I 2021-08-10 10:42:22,430] Trial 65 finished with value: -7.850741711588719 and parameters: {'feature_pre_filter': True, 'lambda_l1': 0.7968838695665004, 'lambda_l2': 2.6441178135645913, 'num_leaves': 43, 'num_trees': 862, 'learning_rate': 0.03595986977731077, 'feature_fraction': 0.38440563115154813, 'min_data_in_leaf': 100}. Best is trial 31 with value: -7.8462917069433304.
[I 2021-08-10 10:42:41,989] Trial 66 finished with value: -8.001021021398445 and parameters: {'feature_pre_filter': True, 'lambda_l1': 0.607610266299393, 'lambda_l2': 3.0169029599294013, 'num_leaves': 42, 'num_trees': 912, 'learning_rate': 0.

[I 2021-08-10 10:51:14,333] Trial 85 finished with value: -7.847152810787644 and parameters: {'feature_pre_filter': True, 'lambda_l1': 0.7613445989468319, 'lambda_l2': 4.915185464586775, 'num_leaves': 40, 'num_trees': 916, 'learning_rate': 0.020720590552754065, 'feature_fraction': 0.2989728126704623, 'min_data_in_leaf': 98}. Best is trial 31 with value: -7.8462917069433304.
[I 2021-08-10 10:51:36,902] Trial 86 finished with value: -7.898839707040577 and parameters: {'feature_pre_filter': True, 'lambda_l1': 0.8357774898735264, 'lambda_l2': 4.4142622410446855, 'num_leaves': 40, 'num_trees': 1000, 'learning_rate': 0.09906629175981783, 'feature_fraction': 0.29110643144832193, 'min_data_in_leaf': 98}. Best is trial 31 with value: -7.8462917069433304.
[I 2021-08-10 10:51:57,744] Trial 87 finished with value: -7.849951861120741 and parameters: {'feature_pre_filter': True, 'lambda_l1': 2.529041122815528, 'lambda_l2': 4.609593439939455, 'num_leaves': 34, 'num_trees': 675, 'learning_rate': 0.025

FrozenTrial(number=31, values=[-7.8462917069433304], datetime_start=datetime.datetime(2021, 8, 10, 10, 27, 1, 509400), datetime_complete=datetime.datetime(2021, 8, 10, 10, 27, 26, 319447), params={'feature_pre_filter': True, 'lambda_l1': 0.7053898373564353, 'lambda_l2': 4.4063317137312294, 'num_leaves': 43, 'num_trees': 935, 'learning_rate': 0.022942371593501133, 'feature_fraction': 0.22233124288732053, 'min_data_in_leaf': 100}, distributions={'feature_pre_filter': CategoricalDistribution(choices=(True, False)), 'lambda_l1': UniformDistribution(high=5.0, low=0.0), 'lambda_l2': UniformDistribution(high=5.0, low=0.0), 'num_leaves': IntUniformDistribution(high=50, low=5, step=1), 'num_trees': IntUniformDistribution(high=1000, low=100, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.01), 'feature_fraction': UniformDistribution(high=1.0, low=0.2), 'min_data_in_leaf': IntUniformDistribution(high=100, low=10, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, tria

In [35]:
study.best_params

{'feature_pre_filter': True,
 'lambda_l1': 0.7053898373564353,
 'lambda_l2': 4.4063317137312294,
 'num_leaves': 43,
 'num_trees': 935,
 'learning_rate': 0.022942371593501133,
 'feature_fraction': 0.22233124288732053,
 'min_data_in_leaf': 100}

In [39]:
best_params = {
    'objective': 'rmse',
    'metric': 'rmse',
    'num_threads': 4,
    'lambda_l1': 0.7,
    'lambda_l2': 4.4,
    'num_leaves': 43,
    'num_trees': 1000,
    'learning_rate': 0.02,
    'feature_fraction': 0.25,
    'min_data_in_leaf': 100
}

lgb = gradient_boost(lgbm.LGBMRegressor(**best_params))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   50.0s remaining:  1.3min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   52.7s remaining:   35.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   55.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   55.9s finished


-7.845282244267366


In [11]:
# def objective_cb(trial):
#     params = {
#         'silent': True,
#         'iterations': trial.suggest_int('iterations', 100, 300),
#         'learning_rate': trial.suggest_float('learning_rate', 0.07, 0.13),
#         'depth': trial.suggest_int('depth', 2, 8),
#         'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 2, 10),
#         'random_strength': trial.suggest_int('random_strength', 1, 3),
#     }
    
#     cat = cb.CatBoostRegressor(**params)
    
#     score = cross_val_score(cat, X, y, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=3)
#     rmse = score.mean()
#     return rmse

In [12]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective_cb, n_trials=100)
# print(study.best_trial)

In [23]:
# params = {
#     'learning_rate': 0.11,
#     'depth': 6,
#     'l2_leaf_reg': 8,
# }

# cat = gradient_boost(cb.CatBoostRegressor(silent=True))

-7.8555571200449945


In [46]:
lgb.booster_.save_model('../lgb.txt')